In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses

from json import load

In [2]:
with open("../data/example_build.json","r") as build_file:
    build_config = load(build_file)

In [3]:
inputs = []
train_config = {
    "dataset":None,
    "optimizer":None,
    "model":None,
    "compile":None,
    "train":None
}


for _id,config in build_config.items():
    if config['type'] == 'Input':
        inputs.append(_id)
        
    elif config['type'] == 'Dataset':
        train_config['dataset'] = _id
        
    elif config['type'] == 'Optimizer':
        train_config['optimizer'] = _id
    
build_config['input_nodes'] = inputs
levels = [ set() for i in range(len(build_config))]
def setLevel(node,config,di=0):
    levels[di].add(node)
    if build_config[node]['connections']['outbound']:
        for next_node in build_config[node]['connections']['outbound']: 
            setLevel(next_node,build_config,di+1)


for inp in inputs:
    setLevel(inp,build_config,0)
    
build_config['levels'] = levels
    
levels = [list(level) for level in levels if len(level)]

In [8]:
if train_config['optimizer']:
    opt = build_config[train_config['optimizer']]
    opt['_type'] = 'Adam'
    
    arguments =  build_arguments(opt['arguments'])  
    globals()[opt['id']] = eval(f"""optimizers.{opt['_type']}(
        {arguments}
    )""")
    
if train_config['dataset']:
    data = build_config[train_config['dataset']]
    exec(data['arguments']['dataset']['value']+f"""
globals()['{train_config['dataset']}'] = Dataset()
""")

In [6]:
def build_inbound(inbound:list)->str:
    inbound = [ f"""build_config['train']['{node}']""" for node in inbound ]
    return ( "([" + ", ".join(inbound) + "])" if len(inbound) > 1 else "(" + inbound[0] + ")" ) if len(inbound) else ""

def set_argument(argument,config):
    value = config['value']
    try:
        value = eval(value)
    except:
        pass
    return f"    {argument}={value.__repr__() if config['type']=='str' else value},"
    
def build_arguments(arguments:dict)->str:
    arguments = '\n'.join([ set_argument(arg,cnf) for arg,cnf in arguments.items() ])
    return arguments

def build_input(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    return eval(f"""layers.Input(
{arguments}
)
""")

def build_default(layer,build_config,*args,**kwargs)->str:
    arguments =  build_arguments(layer['arguments'])
    inbound = build_inbound(layer['connections']['inbound'])
    
    return eval(f"""layers.{layer['type']}(
{arguments}
){inbound}
""")

def build_optimizer(layer,build_config):
    arguments =  build_arguments(layer['arguments'])  
    globals()[layer['id']] = eval(f"""optimizers.{opt['_type']}(
        {arguments}
    )""")
    return globals()[layer['id']]


def build_model(layer,build_config,*args,**kwargs)->str:
    in_nodes = [ f"""build_config['train']['{node}']""" for node in build_config['input_nodes']]
    out_nodes = [ f"""build_config['train']['{node}']""" for node in layer['connections']['inbound']]
    return eval(f"""keras.Model(
    [ {', '.join(in_nodes)}, ],
    [ {', '.join(out_nodes)}, ]
)""")

def build_compile(layer,build_config,*args,**kwargs)->str:
    model,*_ = [node for node in layer['connections']['inbound'] if "model" in node]
    eval(f"""
build_config['train']['{model}'].compile(
    optimizer={train_config['optimizer'] if train_config['optimizer'] else "'"+layer['arguments']['optmizer']['value']+"'"},
    loss='{layer['arguments']['loss']['value']}',
)""")
    return True

def build_train(layer,build_config,*args,**kwargs)->str:
    train_config['train'] = layer
    return ""

In [9]:
build_functions = {
    "Input":build_input,
    "default":build_default,
    "Model":build_model,
    "Compile":build_compile,
    "Train":build_train
}

build_config['train'] = {
    
}

for level in levels:
    for layer in level:
        layer = build_config[layer]
        if layer['type'] in build_functions:
            layer_build = build_functions[layer['type']](layer,build_config) 
        else:
            layer_build = build_functions['default'](layer,build_config) 
        build_config['train'][layer['id']] = layer_build
        
build_config['train']

{'input_1': <KerasTensor: shape=(None, 784) dtype=float32 (created by layer 'input_2')>,
 'dense_1': <KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'dense_1')>,
 'batchnormalization_1': <KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'batch_normalization_1')>,
 'activation_1': <KerasTensor: shape=(None, 10) dtype=float32 (created by layer 'activation_1')>,
 'model_1': <tensorflow.python.keras.engine.functional.Functional at 0x2723fad1c70>,
 'compile_1': True,
 'train_1': ''}

In [70]:
if train_config['dataset']:
    data = build_config[train_config['dataset']]
    exec(data['arguments']['dataset']['value']+f"""
globals()['{train_config['dataset']}'] = Dataset()
""")

In [10]:
model = build_config['train']['model_1']

In [11]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                7850      
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)                40        
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 7,890
Trainable params: 7,870
Non-trainable params: 20
_________________________________________________________________


In [12]:
class TfGui(keras.callbacks.Callback):
    batch = None
    logs = []
    def on_batch_end(self,batch,logs=None):
        self.batch = batch
        self.logs.append(logs)
    
    def on_epoch_end(self,epoch,logs=None):
        print (self.model)
    
tf_gui = TfGui()

In [13]:
dataset = globals()[train_config['dataset']]

model.fit(
    x=dataset.train_x,
    y=dataset.train_y,
    epochs=1,
    batch_size=32,
    validation_data=(dataset.test_x,dataset.test_y) if dataset.validate else None,
    callbacks=[tf_gui]
)

1875/1875 [==============================] - 11s 5ms/step - loss: 0.9047 - val_loss: 0.4415


In [14]:
tf_gui.logs

[{'loss': 2.6443023681640625},
 {'loss': 2.620944023132324},
 {'loss': 2.537588357925415},
 {'loss': 2.467247486114502},
 {'loss': 2.4335193634033203},
 {'loss': 2.434063673019409},
 {'loss': 2.3750271797180176},
 {'loss': 2.3118226528167725},
 {'loss': 2.283987283706665},
 {'loss': 2.245131015777588},
 {'loss': 2.2055628299713135},
 {'loss': 2.1817219257354736},
 {'loss': 2.1609270572662354},
 {'loss': 2.1279985904693604},
 {'loss': 2.1032156944274902},
 {'loss': 2.073552370071411},
 {'loss': 2.0473995208740234},
 {'loss': 2.0338990688323975},
 {'loss': 2.008805513381958},
 {'loss': 1.986183762550354},
 {'loss': 1.9582105875015259},
 {'loss': 1.929540991783142},
 {'loss': 1.9078716039657593},
 {'loss': 1.887444019317627},
 {'loss': 1.8732386827468872},
 {'loss': 1.8519039154052734},
 {'loss': 1.8398371934890747},
 {'loss': 1.828436017036438},
 {'loss': 1.8177210092544556},
 {'loss': 1.8035147190093994},
 {'loss': 1.790029764175415},
 {'loss': 1.7757792472839355},
 {'loss': 1.754375219